In [1]:
# Colab settings
!wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00363/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip" -O "Dataset.zip" -c
!unzip Dataset.zip
!git clone https://github.com/AlinaGalenina/ML_HOMEWORK1.git
from ML_HOMEWORK1.src.models import LinearRegressionWithGd
from ML_HOMEWORK1.src.common import *

--2020-10-09 10:29:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00363/Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19055526 (18M) [application/x-httpd-php]
Saving to: ‘Dataset.zip’

Dataset.zip         100%[===================>]  18.17M  47.0MB/s    in 0.4s    

2020-10-09 10:29:55 (47.0 MB/s) - ‘Dataset.zip’ saved [19055526/19055526]

Archive:  Dataset.zip
   creating: Dataset/
  inflating: Dataset/.DS_Store       
   creating: __MACOSX/
   creating: __MACOSX/Dataset/
  inflating: __MACOSX/Dataset/._.DS_Store  
  inflating: Dataset/Catagory_File - Feature 4.pdf  
  inflating: __MACOSX/Dataset/._Catagory_File - Feature 4.pdf  
   creating: Dataset/Testing/
  inflating: Dataset/Testing/.DS_Store  
   creating: __MACOSX/Dataset/Testing/
  inflating: __MACOSX/Dataset/Testing/._.DS_Store  
  

In [2]:
#import part 
import matplotlib.pyplot as plt
from IPython import display
# from src.models import LinearRegressionWithGd
# from src.common import *
import pandas as pd
path = "Dataset/Training/"

In [3]:
# configuration
num_folds = 5
epochs = 1000
learning_rate = 1e-3

lrg = LinearRegressionWithGd()
lrg.learning_rate = learning_rate

df_1, df_2 = folding(path)
np.random.shuffle(df_1)

### Training

In [4]:
features = []
bias = []
stats = []
losses = [] #loss per fold
for i in range(num_folds):
    train, test = cv_loo(df_1, num_folds, i)

    X_train = normalized(train.T[0:53].T)
    X_test = normalized(test.T[0:53].T)

    Y_train = train.T[53].T
    Y_train = np.expand_dims(Y_train, axis=-1)

    Y_test = test.T[53].T
    Y_test = np.expand_dims(Y_test, axis=-1)

    loss = lrg.train(epochs, X_train, Y_train)
    stat = lrg.validate(X_test, Y_test)
    losses.append(loss)
    stats.append(stat)
    features.append(lrg.betas)
    bias.append(*lrg.bias)
    

mean_stats = np.sum(stats, axis=0) / num_folds
print("mse_val is {}, r2_val is {}, rmse is {}, r2/rmse {}".format(
    mean_stats[0], mean_stats[1], mean_stats[2], mean_stats[3]))


  0%|          | 0/1000 [00:00<?, ?it/s]

val loss is 815.0861201878174, R2 is 0.3342122565945229, RMSE is 28.549713136699246, R2/RMSE is 0.011706326259541626


  0%|          | 0/1000 [00:00<?, ?it/s]

val loss is 814.1856398713746, R2 is 0.3525371902216786, RMSE is 28.533938386969552, R2/RMSE is 0.012355013368314764


  0%|          | 0/1000 [00:00<?, ?it/s]

val loss is 627.7389012177867, R2 is 0.33935017103871956, RMSE is 25.05471814285259, R2/RMSE is 0.013544361948271475


Loss: 770.5777:   0%|          | 0/1000 [00:00<?, ?it/s]

val loss is 841.0681332206946, R2 is 0.3011617185019818, RMSE is 29.001174686910435, R2/RMSE is 0.010384466207084707


Loss: 768.7667: 100%|██████████| 1000/1000 [01:28<00:00, 11.29it/s]

val loss is 804.3312162812123, R2 is 0.35447175899789063, RMSE is 28.36073370491695, R2/RMSE is 0.012498680841124899
mse_val is 780.4820021557771, r2_val is 0.33634661907095864, rmse is 27.90005561166975, r2/rmse 0.012097769724867494


### Features

In [5]:
columns = ["bias"] + [f"f{f+1}" for f in range(len(features[0]))]
index = [f"Fold {i+1}" for i, _ in enumerate(features)] + ["mean"] + ["std"]

bias_np = np.expand_dims(np.squeeze(bias), 1)
features_np = np.squeeze(features, axis=2)
features_np = np.hstack((bias, features_np))
features_mean = np.mean(features_np, axis=0)
features_std = np.std(features_np, axis=0)
df = pd.DataFrame(np.vstack((features_np, features_mean, features_std )),
                  columns=columns,
                  index=index)

display.display(df)    

,bias,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53
Fold 1,7.136941,0.185110,-0.299896,-1.057962,0.124850,-0.419302,1.329027,0.305646,-1.020492,1.223411,1.344457,-0.721460,1.771422,2.089555,0.058928,0.035471,0.125288,2.039689,1.494012,2.083686,-0.732738,0.004617,-0.763804,-1.208590,0.619200,2.109216,-0.484712,0.862130,-0.206261,-0.220517,-1.298851,10.729637,3.098095,-0.243063,6.605762,-3.781700,-0.045973,1.783663,0.420664,0.882063,0.380547,0.444660,0.736014,0.970807,0.561308,0.559321,0.359954,0.575563,0.560355,0.524294,0.698684,0.025518,0.289367,0.290182
Fold 2,7.111789,-0.005431,-0.265554,-0.840793,0.117290,-0.777241,1.651743,0.314994,-0.603802,1.328492,1.588069,-0.788606,2.786044,2.710873,-0.654212,0.007950,0.296696,3.315342,1.840597,2.907985,-1.106870,-0.538501,-1.614195,-1.948948,-0.428119,1.865331,-0.350645,0.430489,-0.565846,-0.694274,-1.442219,10.606265,3.278069,-0.930272,6.366846,-3.828449,-0.013448,1.856002,0.420664,0.889457,0.406890,0.471709,0.671438,1.026713,0.538785,0.543396,0.354412,0.516595,0.630630,0.445024,0.699001,0.063803,0.263599,0.343994
Fold 3,7.300432,0.366389,-0.208590,-1.351914,0.138106,-0.574418,2.045702,0.394304,0.046195,1.569029,1.227233,-0.683702,3.536971,2.434228,-0.976425,0.025827,0.241630,4.090909,1.279534,3.438959,-0.923793,-0.855821,-2.349046,-2.383711,-1.213593,1.772242,0.050610,0.671026,-0.424291,-1.262560,-1.154149,11.170937,3.384023,-1.093454,6.732099,-3.833361,0.023114,1.907073,0.420664,0.851980,0.429590,0.519233,0.744217,1.054085,0.481129,0.418019,0.367461,0.653399,0.620195,0.446282,0.667233,-0.031059,0.237672,0.374282
Fold 4,7.165999,-0.153170,-0.222812,-0.832026,0.086321,-0.815534,2.288807,0.272073,0.537406,1.886877,1.557033,-0.560555,4.216170,2.189039,-1.294394,0.033343,-0.157813,4.874686,1.518908,3.976860,-1.185097,-1.222901,-3.281621,-2.906262,-1.851697,1.749636,0.369906,0.615788,-0.375947,-1.636153,-1.070690,11.090436,3.383557,-1.251294,6.668712,-3.710278,0.025932,1.937281,0.420664,0.876298,0.342770,0.464466,0.701336,0.943522,0.553655,0.562334,0.445473,0.458177,0.619213,0.466158,0.680372,0.062155,0.307476,0.369224
Fold 5,7.127519,0.206082,-0.164033,-1.050323,0.151777,-0.401194,2.591929,0.204664,1.281991,2.261503,0.488402,-0.730520,4.743321,1.299360,-1.480348,0.105879,-0.157206,5.223604,0.946131,4.277775,-0.787953,-1.285339,-4.101950,-3.323773,-2.300849,1.416653,0.382753,1.418132,-1.106420,-2.096310,-0.613455,10.966587,3.402910,-1.422449,6.555430,-3.773642,0.022698,1.917536,0.420664,0.874998,0.364687,0.481317,0.732507,1.027102,0.529488,0.509527,0.365535,0.539790,0.586741,0.509414,0.701979,0.008572,0.251292,0.367736
mean,7.168536,0.119796,-0.232177,-1.026603,0.123669,-0.597538,1.981442,0.298336,0.048260,1.653862,1.241039,-0.696969,3.410786,2.144611,-0.869290,0.041694,0.069719,3.908846,1.415837,3.337053,-0.947290,-0.779589,-2.422123,-2.354257,-1.035011,1.782616,-0.006418,0.799513,-0.535753,-1.181963,-1.115873,10.912773,3.309331,-0.988107,6.585770,-3.785486,0.002465,1.880311,0.420664,0.874959,0.384897,0.476277,0.717102,1.004446,0.532873,0.518520,0.378567,0.548705,0.603427,0.478234,0.689454,0.025798,0.269881,0.349083
std,0.068270,0.180395,0.046877,0.189610,0.022070,0.173591,0.448474,0.061652,0.815938,0.379967,0.399407,0.076036,1.050527,0.474128,0.542850,0.033522,0.193611,1.143729,0.295418,0.782269,0.175494,0.475652,1.180564,0.737884,1.039802,0.222995,0.358771,0.338542,0.307601,0.664986,0.281394,0.213820,0.114406,0.406806,0.124531,0.044607,0.028235,0.055280,0.000000,0.012574,0.030605,0.024620,0.027070,0.040784,0.028158,0.053637,0.033763,0.064789,0.026092,0.032752,0.013487,0.035484,0.025359,0.031253


### Stats

In [6]:
stats_np = np.array(stats)
losses_np = np.expand_dims(np.array(losses), 1)
columns = ["mse_train"] + ["mse_val"] + ["r2_val"] + ["rmse_val"] + ["r2/rmse_val"]
index = [f"Fold {i+1}" for i, _ in enumerate(stats_np)]  + ["mean"] + ["std"]
all_stats = np.hstack((losses_np, stats_np))
all_stats_mean = np.mean(all_stats, axis=0)
all_stats_std = np.std(all_stats, axis=0)
df = pd.DataFrame(np.vstack((all_stats, all_stats_mean, all_stats_std)),
                  columns=columns,
                  index=index)

display.display(df) 

,mse_train,mse_val,r2_val,rmse_val,r2/rmse_val
Fold 1,774.595714,815.086120,0.334212,28.549713,0.011706
Fold 2,767.811631,814.185640,0.352537,28.533938,0.012355
Fold 3,813.585105,627.738901,0.339350,25.054718,0.013544
Fold 4,759.779296,841.068133,0.301162,29.001175,0.010384
Fold 5,769.035840,804.331216,0.354472,28.360734,0.012499
mean,776.961517,780.482002,0.336347,27.900056,0.012098
std,18.913750,77.334726,0.019195,1.438367,0.001040


MD
